## Python Scope Rules

* most languages have the concept of scopes like private, public, protected
    * a class method/field marked private can't be accessed outside the class
    * a public member is accessible from everywhere
    * a protected member is accessible in the class and subclasses but not outside

#### Python doesn't have any scope rules!

* there is no scope definition like private/public/protected
* all class fields and methods are always **public**
    * they can be accessed from anywhere

### Python Coding Convention

* python community recommends adding a single underscore prefix to a field/method to mark it private or for internal use only
* it changes NOTHING. the field/method can still be acessed from anywhere
* It is recommended that outsiders shouldn't try to access it directly
    * it's a convention

#### Let us create an Employee class

In [1]:
class Employee:
    def __init__(self, id, name, salary):
        self._id=id
        self._name=name
        self._salary=salary

    def id(self): return self._id

    def work(self): 
        print(f'{self._name} works as employee #{self.id}')

    def __str__(self): 
        return f"Employee(Id={self._id}\tName={self._name}\tSalary={self._salary})"

In [2]:
e=Employee(1,'Sanjay',20000)
print(e)

Employee(Id=1	Name=Sanjay	Salary=20000)


In [3]:
print(e._id) # works. but we shouldn't access it directly

#instead we should use official methdo if provided

print(e.id()) # good code

1
1


### A more safer private (internal)

* A field with double underscore prefix will be more safe
* It can't be easily modified.
* Here salary is more private
    * it is difficult to change

In [4]:
class Employee:
    def __init__(self, id, name, salary):
        self._id=id
        self._name=name
        self.__salary=salary

    def id(self): return self._id

    def work(self): 
        print(f'{self._name} works as employee #{self.id}')

    def salary(self): return self.__salary

    def __str__(self): 
        return f"Employee(Id={self._id}\tName={self._name}\tSalary={self.__salary})"

In [5]:
e=Employee(1,'Sanjay',20000)
print(e)
print(e.salary())

Employee(Id=1	Name=Sanjay	Salary=20000)
20000


#### we may try to modify the value

* it may even appear value has changed

In [6]:
e.__salary=500000
print(e.__salary)

500000


##### But value remains unchanged if we look through other official methods


In [7]:
print(e)
print(e.salary())
print(e.__salary)

Employee(Id=1	Name=Sanjay	Salary=20000)
20000
500000


### What happened?

* when we declare a member with double underscore prefix python internall saves it differently
* It changes the name internally to **\_ClassName__fieldName**

```python
class Employee:
    def __init__(self, id,name, salary):
        self.__salary=salary # self._Employee__salary=salary

    def salary(self):
        return __salary # changes to self._Employee__salary
```

* But when we try to modify it from outside
    * this change doesn't happen
    * A new field is created with this name

In [8]:
e2=Employee(2,'Prabhat',10000)
print(e2)

Employee(Id=2	Name=Prabhat	Salary=10000)


In [9]:
def salary_info(e):
    salary_related =[ x for x in dir(e2) if "salary" in x]
    print(salary_related)

In [10]:
salary_info(e2)

['_Employee__salary', 'salary']


In [11]:
e2.__salary=200000
salary_info(e2)

['_Employee__salary', '__salary', 'salary']


In [12]:
print(e2)

Employee(Id=2	Name=Prabhat	Salary=10000)


#### since we know it, we can break it

In [13]:
e2._Employee__salary=200000
print(e2)

Employee(Id=2	Name=Prabhat	Salary=200000)


### What is the problem in the below code?

In [14]:
e1 = Employee(1,"Sanjay",20000)
e2 = Employee(1,"Prabhat",30000)

print(e1)
print(e2)

Employee(Id=1	Name=Sanjay	Salary=20000)
Employee(Id=1	Name=Prabhat	Salary=30000)


#### Problem

* we can have multiple employees with same id
* ideally this shouldn't be allowed.
* but each object is different and can't look into other's attribute
* how do I makes sure each of them will get a uniquely different id?


### Solution #1 Auto generate Id using a global variable

In [17]:
lastId=0
class Employee:
    def __init__(self, name, salary):
        global lastId
        lastId+=1
        self._id=lastId
        self._name=name
        self._salary=salary

    def __str__(self):
        return f"Employee(Id={self._id}\tName={self._name}\tSalary={self._salary})"

In [18]:
e1=Employee("Sanjay",50000)
e2=Employee("Prabhat",20000)

print(e1)
print(e2)

Employee(Id=1	Name=Sanjay	Salary=50000)
Employee(Id=2	Name=Prabhat	Salary=20000)


#### Problem

* we may have diffrent types of object that may need auto gerated ids
    * example:
        * Department
        * Project
        * Invoice

    * we can't have multiple **lastId** for each of them globally

##### Solution #2 : declare prefixed id
 * example
    * lastEmployeeId
    * lastDepartmentId

##### Solution #3: delcare lastId at the module level




### Solution #4  Create class level field lastId

* just like method (at same indent) we can have class level fields
* this field will be accessible in every object using **self.** or **classname.**
* it will be shared member and not owned by any particular object

In [20]:

class Employee:
    _lastId=0 #shared memeber
    def __init__(self, name, salary):        
        Employee._lastId+=1 #accessing lastId using class referece
        self._id=self._lastId #accessing last is using object/self reference
        self._name=name
        self._salary=salary

    def getLastId(self):
        return self._lastId

    def employeeCount():
        return Employee.lastId

    def __str__(self):
        return f"Employee(Id={self._id}\tName={self._name}\tSalary={self._salary})"

In [21]:
### Happy Case
e1=Employee("Sanjay",20000)
e2=Employee("Prabhat",50000)

print(e1)
print(e2)


Employee(Id=1	Name=Sanjay	Salary=20000)
Employee(Id=2	Name=Prabhat	Salary=50000)


In [22]:
# _lastId can be accessed using class reference
print(Employee._lastId)

# or using object reference
print(e1._lastId)
print(e2._lastId)


2
2
2


In [23]:
e3=Employee("Amit",30000)
print(Employee._lastId)
print(e1._lastId)
print(e2._lastId)
print(e3._lastId)

3
3
3
3


### But the shared should never be modified by object reference

* a shared field should modified only by class reference and NOT by object reference

* if we modify the shared filed using object reference, it creates a new field for that object only. It will have no effect on other objects or shared filed


### How fields are resolved

### When we try to access **obj.something**

1. python first checks if a property **something** is present in current object or not
    * if found, you get it

2. in **something** is not defined in current object, it checks if it is present in class or not.
    * if found, you get it

3. It raises exception


##### Where do I see object properties

* every object has \_\_dict\_\_ property that contains properties of current object
* shared memebers are not present here

In [30]:
print(e1.__dict__)
print(type(e1).__dict__)

{'_id': 1, '_name': 'Sanjay', '_salary': 20000}
{'__module__': '__main__', '_lastId': 3, '__init__': <function Employee.__init__ at 0x0000022F03D5CE00>, 'getLastId': <function Employee.getLastId at 0x0000022F03D5D120>, 'employeeCount': <function Employee.employeeCount at 0x0000022F03D5D1C0>, '__str__': <function Employee.__str__ at 0x0000022F03D5D260>, '__dict__': <attribute '__dict__' of 'Employee' objects>, '__weakref__': <attribute '__weakref__' of 'Employee' objects>, '__doc__': None}


In [31]:
print(e1._lastId) #searches in e1.__dict__ and then in type(e1).__dict__

3


### When we try to modify the property

* when we try to modify a property  **obj.something= value** 
    * python simply adds this value to current object
    * it doesn't check for shared member

In [34]:
print(e1.__dict__)
e1._lastId=100
print(e1.__dict__)
print(Employee.__dict__)

{'_id': 1, '_name': 'Sanjay', '_salary': 20000, '_lastId': 100}
{'_id': 1, '_name': 'Sanjay', '_salary': 20000, '_lastId': 100}
{'__module__': '__main__', '_lastId': 3, '__init__': <function Employee.__init__ at 0x0000022F03D5CE00>, 'getLastId': <function Employee.getLastId at 0x0000022F03D5D120>, 'employeeCount': <function Employee.employeeCount at 0x0000022F03D5D1C0>, '__str__': <function Employee.__str__ at 0x0000022F03D5D260>, '__dict__': <attribute '__dict__' of 'Employee' objects>, '__weakref__': <attribute '__weakref__' of 'Employee' objects>, '__doc__': None}


#### What happens next?

* Employee._lastId and other_object._lastId will be same
* For object **el** _lastId will be different and cause confusion


### Best Practices guidelines

* Always access a shared member using class reference and NEVER using object reference

In [35]:

class Employee:
    _lastId=0 #shared memeber
    def __init__(self, name, salary):        
        Employee._lastId+=1 #accessing lastId using class referece
        self._id=Employee._lastId #accessing last is using object/self reference
        self._name=name
        self._salary=salary

    def getLastId(self):
        return Employee._lastId

    def employeeCount():
        return Employee.lastId

    def __str__(self):
        return f"Employee(Id={self._id}\tName={self._name}\tSalary={self._salary})"

### Shared Members are often called static

* most other programming language refers these shared members a static